<a href="https://colab.research.google.com/github/verityw/manipulation-final-project/blob/main/DataGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve
import time

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='latest', drake_build='continuous')

# Determine if this notebook is currently running as a notebook or a unit test.
from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# Setup rendering (with xvfb), if necessary:
import os
if 'google.colab' in sys.modules and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# Start two meshcat servers (one for 2d) to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc_2d, zmq_url_2d, web_url_2d = start_zmq_server_as_subprocess(server_args=server_args)
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

# Imports
import numpy as np
from IPython.display import display, HTML
from ipywidgets import Textarea

from pydrake.all import ( 
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, 
    DiagramBuilder, RigidTransform, RotationMatrix, Box,    
    CoulombFriction, FindResourceOrThrow, FixedOffsetFrame, 
    GeometryInstance, MeshcatContactVisualizer, Parser, PlanarJoint,  
    JointIndex, Simulator, ProcessModelDirectives, LoadModelDirectives
)

from functools import partial
import open3d as o3d
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

from pydrake.all import (
    ConnectPlanarSceneGraphVisualizer,
    ConnectDrakeVisualizer, DepthCameraProperties, RgbdSensor,
    RandomGenerator, UniformlyRandomRotationMatrix, RollPitchYaw,
    MakeRenderEngineVtk, RenderEngineVtkParams, Role, UnitInertia, set_log_level
)

from ipywidgets import Dropdown, FloatSlider, Layout
from pydrake.all import (
    Sphere, Cylinder, Box, Capsule, Ellipsoid, SpatialInertia)

import pydrake

#from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
from manipulation.meshcat_utils import draw_open3d_point_cloud, draw_points
from manipulation.open3d_utils import create_open3d_point_cloud
from manipulation.mustard_depth_camera_example import MustardExampleSystem
from manipulation.scenarios import AddRgbdSensors
from manipulation.utils import FindResource

set_log_level("warn");

ycb = [("cracker", "003_cracker_box.sdf"), 
        ("sugar", "004_sugar_box.sdf"), 
        ("soup", "005_tomato_soup_can.sdf"), 
        ("mustard", "006_mustard_bottle.sdf"), 
        ("gelatin", "009_gelatin_box.sdf"), 
        ("meat", "010_potted_meat_can.sdf")]


Cloning into '/opt/manipulation'...

Already on 'master'

/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK

ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.5.3 which is incompatible.
ERROR: nbclient 0.5.1 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.






In [ ]:
import os
from tensorflow import keras
from google.colab import drive
drive.mount('/content/gdrive')

SAMPLES = -1
BATCH_SIZE = 4
EPOCHS = 10
IMG_SHAPE = (480, 640, 4)
POSE_SHAPE = (6,)
VAL_PROPORTION = .1

Mounted at /content/gdrive


In [ ]:
# cd to directory containing this notebook

In [ ]:
def data_gen(objects, val = False, max_iter = 10000):
    """
    Run MCMC to refine initial pose guesses for all objects

    Args:
    objects: List of YCB object indices from ycb = [("cracker", "003_cracker_box.sdf"), ("sugar", "004_sugar_box.sdf"), ("soup", "005_tomato_soup_can.sdf"), ("mustard", "006_mustard_bottle.sdf"), ("gelatin", "009_gelatin_box.sdf"), ("meat", "010_potted_meat_can.sdf")]
    """
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0005)

    parser = Parser(plant)

    parser.AddModelFromFile(FindResourceOrThrow(
        "drake/examples/manipulation_station/models/bin.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("bin_base"))


    rs = np.random.RandomState()  # this is for python
    generator = RandomGenerator(rs.randint(1000))  # this is for c++

    # Only generate a few objects for testing.
    for i in objects:
        object_num = i
        sdf = FindResourceOrThrow("drake/manipulation/models/ycb/sdf/" + ycb[object_num][1])
        parser.AddModelFromFile(sdf, f"object{i}")

    plant.Finalize()

    renderer = "my_renderer"
    scene_graph.AddRenderer(
        renderer, MakeRenderEngineVtk(RenderEngineVtkParams()))
    properties = DepthCameraProperties(width=640,
                                       height=480,
                                       fov_y=np.pi / 4.0,
                                       renderer_name=renderer,
                                       z_near=0.1,
                                       z_far=10.0)
    camera = builder.AddSystem(
        RgbdSensor(parent_id=scene_graph.world_frame_id(),
                   X_PB=RigidTransform(
                       RollPitchYaw(np.pi, 0, np.pi/2.0),
                       [0, 0, .8]),
                   properties=properties,
                   show_window=False))
    camera.set_name("rgbd_sensor")
    builder.Connect(scene_graph.get_query_output_port(),
                    camera.query_object_input_port())
    builder.ExportOutput(camera.color_image_output_port(), "color_image")

    # Note: if you're running this on a local machine, then you can 
    # use drake_visualizer to see the simulation.  (It's too slow to 
    # show the meshes on meshcat).
    vis = ConnectDrakeVisualizer(
        builder, 
        scene_graph
    )
    

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant.GetMyContextFromRoot(context)
    
    if val:
        xd, yd = XVAL_DIR, YVAL_DIR
    else:
        xd, yd = X_DIR, Y_DIR

    for ind in range(max_iter):
        z = 0.07
        theta = np.random.rand() * 2 * np.pi
        for body_index in plant.GetFloatingBaseBodies():
            tf = RigidTransform(
                    rpy=RollPitchYaw(0, 0, theta), #UniformlyRandomRotationMatrix(generator),  
                    p=[rs.uniform(-.12,.12), rs.uniform(-.18, .18), z])
            plant.SetFreeBodyPose(plant_context, 
                                    plant.get_body(body_index),
                                    tf)
            z += 0.1

        simulator.AdvanceTo(1.0 if running_as_notebook else 0.1)
        for body_index in plant.GetFloatingBaseBodies():
            tf = plant.GetFreeBodyPose(plant_context, plant.get_body(body_index))
        color_image = diagram.GetOutputPort("color_image").Eval(context).data
        quaternion = pydrake.common.eigen_geometry.Quaternion_(tf.rotation().matrix())
        
        six_d_pose = np.hstack((tf.translation(), [quaternion.w(), quaternion.x(), quaternion.y(), quaternion.z()]))

        img_path = os.path.join(xd, str(ind) + "img")
        pose_path = os.path.join(yd, str(ind) + "pose")

        np.save(img_path, color_image)
        np.save(pose_path, six_d_pose)
        
        if ind % (max_iter // 10) == 0 and ind != 0:
            print(str(100 * ind / max_iter) + "% complete" )
            

        



In [ ]:
DATA_DIRECTORY = os.path.join(os.getcwd(), "data")
X_DIR, Y_DIR = os.path.join(DATA_DIRECTORY, "x5"), os.path.join(DATA_DIRECTORY, "y5")
#XVAL_DIR, YVAL_DIR = os.path.join(DATA_DIRECTORY, "xval"), os.path.join(DATA_DIRECTORY, "yval")

In [ ]:
for i in [""]:
    print(i)
    #X_DIR, Y_DIR = os.path.join(DATA_DIRECTORY, "x" + str(i)), os.path.join(DATA_DIRECTORY, "y" + str(i))
    XVAL_DIR, YVAL_DIR = os.path.join(DATA_DIRECTORY, "xval" + str(i)), os.path.join(DATA_DIRECTORY, "yval" + str(i))
    data_gen([2], val=True, max_iter=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DrakeDeprecationWarning: Deprecated; due to be removed after 2021-03-01. Please use DepthRenderCamera instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: DrakeDeprecationWarning: Constructs an RgbdSensor in the same way as the above overload, but
using the ``CameraProperties`` portion of ``properties`` for color
(and label) properties, and all of ``properties`` for depth
properties.  /
(Deprecated.)

Deprecated:
    CameraProperties are being deprecated. Please use the RenderCamera
    variant. This will be removed from Drake on or after 2021-03-01.
[2020-11-29 00:23:50.828] [console] [warning] RgbdSensor constructors that take 'CameraProperties' have been deprecated. Please use the RenderCamera variants instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:56: DrakeDeprecationWarning: Extends a Diagram with the required components to interface with
drake_visualizer. This must be calle

10.0% complete
20.0% complete
30.0% complete
40.0% complete
50.0% complete
60.0% complete
70.0% complete
80.0% complete
90.0% complete


In [ ]:
data_gen([2], val=True, max_iter=2000)